In [36]:
# from google.colab import files
# uploaded = files.upload()

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
# get your data from here - "https://www.kaggle.com/datasets/thedevastator/unlock-profits-with-e-commerce-sales-data"

In [38]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/big data/Amazon Sale Report.csv", usecols = ["Date", "Style", "SKU", "Category", "Size", "Qty", "Amount", "ship-city", "ship-state"])
df = df.set_index('Date').sort_index()

In [39]:
df

,Style,SKU,Category,Size,Qty,Amount,ship-city,ship-state
Date,,,,,,,,
03-31-22,JNE3724,JNE3724-KR-L,kurta,L,1,495.00,KOLKATA,WEST BENGAL
03-31-22,SET187,SET187-KR-DH-XL,Set,XL,1,599.00,BENGALURU,KARNATAKA
03-31-22,JNE3383,JNE3383-KR-A-M,kurta,M,0,416.19,GURGAON,HARYANA
03-31-22,JNE3405,JNE3405-KR-L,kurta,L,1,449.00,Nalasopara east,MAHARASHTRA
03-31-22,SET044,SET044-KR-NP-L,Set,L,1,542.00,HYDERABAD,TELANGANA
...,...,...,...,...,...,...,...,...
06-29-22,SET196,SET196-KR-NP-XL,Set,XL,1,668.00,Bangalore,KARNATAKA
06-29-22,SET044,SET044-KR-NP-S,Set,S,1,612.00,PATNA,BIHAR
06-29-22,J0111,J0111-TP-XL,Top,XL,1,423.00,BENGALURU,KARNATAKA


In [40]:
# some data cleaning of converting all categorical cols to numerical
for col in df.columns:
  if df[col].dtype == 'object':
    df[col] = df[col].astype('category').cat.codes + 1

df

,Style,SKU,Category,Size,Qty,Amount,ship-city,ship-state
Date,,,,,,,,
03-31-22,798,4099,9,6,1,495.00,3713,62
03-31-22,1173,5924,6,9,1,599.00,777,24
03-31-22,538,2655,9,7,0,416.19,2616,20
03-31-22,556,2747,9,6,1,449.00,5549,29
03-31-22,1109,5622,6,6,1,542.00,2907,58
...,...,...,...,...,...,...,...,...
06-29-22,1180,5965,6,9,1,668.00,1106,24
06-29-22,1109,5624,6,8,1,612.00,5963,9
06-29-22,196,762,7,9,1,423.00,777,24


In [41]:
df.to_csv('amazon9.csv', index= True)

In [42]:
pip install pyspark

In [43]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('amazon').getOrCreate()

In [44]:
from pyspark.ml.regression import LinearRegression

In [45]:
df = spark.read.csv('amazon9.csv', inferSchema = True, header = True)
df.show()

+--------+-----+----+--------+----+---+------+---------+----------+
|    Date|Style| SKU|Category|Size|Qty|Amount|ship-city|ship-state|
+--------+-----+----+--------+----+---+------+---------+----------+
|03-31-22|  798|4099|       9|   6|  1| 495.0|     3713|        62|
|03-31-22| 1173|5924|       6|   9|  1| 599.0|      777|        24|
|03-31-22|  538|2655|       9|   7|  0|416.19|     2616|        20|
|03-31-22|  556|2747|       9|   6|  1| 449.0|     5549|        29|
|03-31-22| 1109|5622|       6|   6|  1| 542.0|     2907|        58|
|03-31-22|  864|4549|       8|   6|  0|724.76|     8082|        25|
|03-31-22| 1255|6401|       6|   1|  0|  NULL|     3713|        62|
|03-31-22| 1291|6635|       6|   6|  0|  NULL|     2907|        58|
|03-31-22|  378|1797|       6|   7|  1| 519.0|     1212|        24|
|03-31-22| 1328|6888|       6|   8|  1|1165.0|     1272|        29|
|03-31-22|  215| 867|       9|   6|  1| 358.0|     5963|         9|
|03-31-22|  696|3502|       9|  11|  1| 417.0|  

In [46]:
df = df.dropna()

In [47]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Style: integer (nullable = true)
 |-- SKU: integer (nullable = true)
 |-- Category: integer (nullable = true)
 |-- Size: integer (nullable = true)
 |-- Qty: integer (nullable = true)
 |-- Amount: double (nullable = true)
 |-- ship-city: integer (nullable = true)
 |-- ship-state: integer (nullable = true)



In [48]:
# normally in sk learn you give columns individually but in spark you make of list of all independent columns and give for that we need vector assembler

In [49]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [50]:
featureassembler = VectorAssembler(inputCols = ['Style', 'SKU', 'Category', 'Size', 'Qty', 'ship-city', 'ship-state'], outputCol='Independent Features')
output = featureassembler.transform(df)
output.show()

+--------+-----+----+--------+----+---+------+---------+----------+--------------------+
|    Date|Style| SKU|Category|Size|Qty|Amount|ship-city|ship-state|Independent Features|
+--------+-----+----+--------+----+---+------+---------+----------+--------------------+
|03-31-22|  798|4099|       9|   6|  1| 495.0|     3713|        62|[798.0,4099.0,9.0...|
|03-31-22| 1173|5924|       6|   9|  1| 599.0|      777|        24|[1173.0,5924.0,6....|
|03-31-22|  538|2655|       9|   7|  0|416.19|     2616|        20|[538.0,2655.0,9.0...|
|03-31-22|  556|2747|       9|   6|  1| 449.0|     5549|        29|[556.0,2747.0,9.0...|
|03-31-22| 1109|5622|       6|   6|  1| 542.0|     2907|        58|[1109.0,5622.0,6....|
|03-31-22|  864|4549|       8|   6|  0|724.76|     8082|        25|[864.0,4549.0,8.0...|
|03-31-22|  378|1797|       6|   7|  1| 519.0|     1212|        24|[378.0,1797.0,6.0...|
|03-31-22| 1328|6888|       6|   8|  1|1165.0|     1272|        29|[1328.0,6888.0,6....|
|03-31-22|  215| 867|

In [51]:
output.select('Independent Features').show()

+--------------------+
|Independent Features|
+--------------------+
|[798.0,4099.0,9.0...|
|[1173.0,5924.0,6....|
|[538.0,2655.0,9.0...|
|[556.0,2747.0,9.0...|
|[1109.0,5622.0,6....|
|[864.0,4549.0,8.0...|
|[378.0,1797.0,6.0...|
|[1328.0,6888.0,6....|
|[215.0,867.0,9.0,...|
|[696.0,3502.0,9.0...|
|[112.0,348.0,6.0,...|
|[1309.0,6757.0,6....|
|[870.0,4607.0,9.0...|
|[489.0,2393.0,9.0...|
|[864.0,4551.0,8.0...|
|[720.0,3654.0,9.0...|
|[796.0,4091.0,9.0...|
|[374.0,1769.0,8.0...|
|[557.0,2759.0,9.0...|
|[788.0,4038.0,7.0...|
+--------------------+
only showing top 20 rows



In [52]:
finalized_data = output.select('Independent Features','Amount')
finalized_data.show()

+--------------------+------+
|Independent Features|Amount|
+--------------------+------+
|[798.0,4099.0,9.0...| 495.0|
|[1173.0,5924.0,6....| 599.0|
|[538.0,2655.0,9.0...|416.19|
|[556.0,2747.0,9.0...| 449.0|
|[1109.0,5622.0,6....| 542.0|
|[864.0,4549.0,8.0...|724.76|
|[378.0,1797.0,6.0...| 519.0|
|[1328.0,6888.0,6....|1165.0|
|[215.0,867.0,9.0,...| 358.0|
|[696.0,3502.0,9.0...| 417.0|
|[112.0,348.0,6.0,...| 688.0|
|[1309.0,6757.0,6....| 657.0|
|[870.0,4607.0,9.0...| 453.0|
|[489.0,2393.0,9.0...| 599.0|
|[864.0,4551.0,8.0...| 761.0|
|[720.0,3654.0,9.0...| 453.0|
|[796.0,4091.0,9.0...| 322.0|
|[374.0,1769.0,8.0...|1099.0|
|[557.0,2759.0,9.0...| 248.0|
|[788.0,4038.0,7.0...| 354.0|
+--------------------+------+
only showing top 20 rows



In [53]:
# Creating my regressor model
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])

regressor = LinearRegression(featuresCol='Independent Features', labelCol='Amount')
regressor_model = regressor.fit(train_data)

In [54]:
regressor_model.coefficients, regressor_model.intercept

(DenseVector([-0.6351, 0.1219, -87.9812, -2.3018, 89.1365, 0.0004, 0.0759]),
 1239.2887957002984)

In [55]:
pred_results = regressor_model.evaluate(test_data)
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Amount|        prediction|
+--------------------+------+------------------+
|[1.0,3.0,2.0,11.0...| 229.0| 1134.612946233894|
|[2.0,4.0,2.0,7.0,...| 301.0|1142.0682767452806|
|[2.0,5.0,2.0,8.0,...| 229.0|1135.5529116342634|
|[4.0,11.0,2.0,9.0...|306.67|1046.3848432263965|
|[5.0,13.0,2.0,8.0...| 241.0|1134.7720406797846|
|[8.0,26.0,2.0,11....| 218.1| 1041.026038406429|
|[8.0,26.0,2.0,11....| 310.0|1130.2052222129598|
|[8.0,26.0,2.0,11....| 229.0| 1130.892244906216|
|[8.0,26.0,2.0,11....| 301.0| 1131.028253905276|
|[9.0,27.0,2.0,9.0...| 241.0| 1135.270153934197|
|[10.0,29.0,2.0,7....|286.67| 1050.897198975818|
|[11.0,31.0,2.0,6....| 301.0|1138.1221175473645|
|[11.0,31.0,2.0,6....| 301.0|1137.9535459077301|
|[11.0,31.0,2.0,6....| 602.0|1229.2109647405023|
|[11.0,34.0,2.0,11...| 301.0| 1128.625242722621|
|[13.0,42.0,2.0,8....| 229.0|1132.4766581682043|
|[15.0,45.0,1.0,5....| 419.0| 1229.820841236027|
|[15.0,46.0,1.0,5...

In [56]:
# well its very bad :-)